In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install python-Levenshtein

In [3]:
pip install fuzzywuzzy

In [0]:
import pandas as pd, numpy as np
from fuzzywuzzy import process

In [0]:
correct_cities = pd.read_csv('/content/drive/My Drive/Salesken.ai/Correct_cities.csv')
misspelt_cities = pd.read_csv('/content/drive/My Drive/Salesken.ai/Misspelt_cities.csv')

In [0]:
correct_cities = correct_cities.sort_values('country')
misspelt_cities = misspelt_cities.sort_values('country')

In [7]:
correct_cities.tail()

,name,country,id
22993,Victoria Falls,Zimbabwe,879431
22992,Zvishavane,Zimbabwe,878549
23016,Epworth,Zimbabwe,1085510
23003,Kariba,Zimbabwe,889215
23017,Chitungwiza,Zimbabwe,1106542


In [8]:
misspelt_cities.tail()

,misspelt_name,country
11549,Harore,Zimbabwe
5032,Chkpinge,Zimbabwe
16948,Shirugwi,Zimbabwe
4537,Karox,Zimbabwe
12962,Butawayo,Zimbabwe


In [9]:
misspelt_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23018 entries, 13609 to 12962
Data columns (total 2 columns):
misspelt_name    23018 non-null object
country          23018 non-null object
dtypes: object(2)
memory usage: 539.5+ KB


In [0]:
correct_cities['name'] = correct_cities['name'].astype('str')
misspelt_cities['misspelt_name'] = misspelt_cities['misspelt_name'].astype('str')

In [0]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=80):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [0]:
def ids(correct_cities, misspelt_cities, threshold):

  total_df = pd.DataFrame(columns=correct_cities.columns)
  countries = correct_cities.country.unique().tolist()

  for country in countries:
    df = fuzzy_merge(correct_cities.loc[correct_cities['country'] == country], misspelt_cities.loc[misspelt_cities['country'] == country], \
                    'name', 'misspelt_name', threshold = 80)
    total_df = total_df.append(df, ignore_index=True, sort = True)
    truncated_df = total_df[['id','matches']]

  return truncated_df

In [13]:
ids(correct_cities, misspelt_cities, 80)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,id,matches
0,1141857,Gardēl
1,1135158,bushk
2,1135689,Kynduz
3,1136469,Khqst
4,1136575,Khulo
...,...,...
23013,879431,Victoria rplls
23014,878549,Ztishavaue
23015,1085510,Esworth
23016,889215,Karika


In [0]:
df1 = pd.DataFrame({'name': ['Bangalore','Karachi','Kolkata'],'country':['India','Pakistan','India'], 'id':[23,45,50]})
df2 = pd.DataFrame({'misspelt_name' : ['Karahffi','Kolkattu','Bengaloe'],'country':['Pakistan','India','India']})

In [15]:
ids(df1, df2, 60)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,id,matches
0,23,Bengaloe
1,50,Kolkattu
2,45,Karahffi
